In [1]:
# Load API_key from .env file
from config_loader import ConfigLoader
config = ConfigLoader()

In [2]:
from typing import Annotated

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.checkpoint.memory import MemorySaver

def select_model():
    choice_map = {
        "1": ("Claude", ChatAnthropic(model="claude-3-5-sonnet-20240620")),
        "2": ("GPT-3", ChatOpenAI(model_name="gpt-3.5-turbo-1106"))
    }

    while (choice := input("Enter 1 for Claude or 2 for GPT-3: ").strip()) not in choice_map:
        print("Invalid choice. Please enter 1 or 2.")

    model_name, llm_instance = choice_map[choice]
    print(f"You selected {model_name}.")
    return llm_instance

memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]    

graph_builder = StateGraph(State)

#Add New function; Tools
tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = select_model()
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

#This is new
graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["tools"],
    # interrupt_after=["tools"]
)

Enter 1 for Claude or 2 for GPT-3:  1


You selected Claude.


In [3]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_ZJGbXR86tDxtccdcZJWTdwum)
 Call ID: call_ZJGbXR86tDxtccdcZJWTdwum
  Args:
    query: LangGraph


In [3]:
#현재 graph의 어느 위치에 있는지 확인
snapshot = graph.get_state(config)
#자이제 다음단계로 가보자
snapshot.next

TypeError: 'ConfigLoader' object is not subscriptable

In [5]:
existing_message = snapshot.values["messages"][-1]
print(existing_message)
existing_message.tool_calls

content='' additional_kwargs={'tool_calls': [{'id': 'call_ZJGbXR86tDxtccdcZJWTdwum', 'function': {'arguments': '{"query":"LangGraph"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 97, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_49836e94c4', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-b343671c-ccf4-446b-acc4-e32e42e6a4dd-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LangGraph'}, 'id': 'call_ZJGbXR86tDxtccdcZJWTdwum', 'type': 'tool_call'}] usage_metadata={'input_tokens': 97, 'output_tokens': 19, 'total_tokens': 116, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audi

[{'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph'},
  'id': 'call_ZJGbXR86tDxtccdcZJWTdwum',
  'type': 'tool_call'}]

In [9]:
#멈춰있는데 아래의 명령어를 듣고 재시작-1
# exisiting_message를 보면 {'query': 'LangGraph for language models'} 로 LLM이 뽑은 쿼리가 합리적으로 느껴짐 그냥 두자
# None이라는 명령어를 사람이 줘서 다음으로 갈수있게된것!
events = graph.stream(None, config, stream_mode="values")

#멈춰있는데 아래의 명령어를 듣고 재시작-2
# 괜히 한번 바꿔보자/나는 이코드 잘안됨 개념이해로만 유트브로봄
#user_input_2 = "Please answer for the question in Korean."
#events = graph.stream({"messages": [("user", user_input_2)]}, config, stream_mode="values")


In [10]:
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Please answer for the question in Korean.


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_ZJGbXR86tDxtccdcZJWTdwum", 'type': 'invalid_request_error', 'param': 'messages.[2].role', 'code': None}}